<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [141]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os, sys

In [142]:
tr_mcc_codes = pd.read_csv(filepath_or_buffer="data/tr_mcc_codes.csv", sep = ';')
tr_types = pd.read_csv(filepath_or_buffer="data/tr_types.csv", sep=';')
transactions = pd.read_csv(filepath_or_buffer="data/transactions.csv", sep=",", nrows=1000000)
customers_gender_train = pd.read_csv(filepath_or_buffer="data/gender_train.csv", sep=",")

In [153]:
df = pd.merge(transactions, customers_gender_train, how = "left")
df = pd.merge(df, tr_mcc_codes, how="inner")
df = pd.merge(df, tr_types, how="inner")
tr_day = df.tr_datetime.str.split(n = 1, expand = True)
df["tr_day"] = tr_day[0].astype(int)
# df["tr_time"] = tr_day[1]
df = df.drop(["tr_datetime", "mcc_description", "tr_description", "term_id"], axis=1)
df = df[df.gender.notnull()]
df.head()

,customer_id,mcc_code,tr_type,amount,gender,tr_day
0,39026145,4814,1030,-2245.92,1.0,0
1,39026145,4814,1030,-5614.79,1.0,6
2,39026145,4814,1030,-1122.96,1.0,8
3,39026145,4814,1030,-2245.92,1.0,11
4,39026145,4814,1030,-2245.92,1.0,11


In [144]:
df_copy = df[df.amount<0].groupby("tr_day").amount.sum()
df_copy.index = df_copy.index.astype(int)
df_copy.sort_index()
# df.copy.plot(figsize=(10, 5))

tr_day
0     -66440564.28
1     -36318167.99
2     -57511150.68
3     -69148579.78
4     -77088269.80
          ...     
452   -85771613.52
453   -71216809.80
454   -53757244.73
455   -81760922.17
456   -64930480.51
Name: amount, Length: 457, dtype: float64

In [154]:
features = df.loc[:, df.columns != 'gender'].values[:, 1:]
labels = df.loc[:, 'gender'].values

In [155]:
features

array([[ 4.81400000e+03,  1.03000000e+03, -2.24592000e+03,
         0.00000000e+00],
       [ 4.81400000e+03,  1.03000000e+03, -5.61479000e+03,
         6.00000000e+00],
       [ 4.81400000e+03,  1.03000000e+03, -1.12296000e+03,
         8.00000000e+00],
       ...,
       [ 3.00000000e+03,  6.00000000e+03,  1.49164742e+06,
         2.61000000e+02],
       [ 3.00000000e+03,  6.00000000e+03,  1.49838517e+06,
         2.63000000e+02],
       [ 3.00000000e+03,  6.00000000e+03,  5.70417690e+05,
         2.70000000e+02]])

In [156]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1, 1))
X = scaler.fit_transform(features)
Y = labels

In [148]:
Y

array([1., 1., 1., ..., 0., 0., 0.])

In [157]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

In [158]:
model = XGBClassifier()
model.fit(X_train, Y_train)

[16:58:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [160]:
Y_hat = [round(yhat) for yhat in model.predict(X_test)]
print(accuracy_score(Y_test, Y_hat)) # Test set accuracy
Y_hat = [round(yhat) for yhat in model.predict(X)]
print(accuracy_score(Y, Y_hat)) # Full set accuracy

0.6376490312025006
0.6474821942188287


In [ ]:
print(model.feature_importances_)

In [ ]:
from matplotlib import pyplot
from xgboost import plot_importance
plot_importance(model)
pyplot.show()